# **Model**

In [107]:
## 모듈 임포트
import numpy as np 
import pandas as pd
import warnings
warnings.simplefilter("ignore")

from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split  ## train test 
from sklearn.preprocessing import LabelEncoder      ## object 정수 인코딩
from sklearn.preprocessing import OneHotEncoder     ## object one-hot
from sklearn.preprocessing import StandardScaler    ## 표준 정규 분포 이용

from sklearn.linear_model import LinearRegression ## 회귀 분석
from sklearn.svm import SVC                       ## svm 분류모델
from sklearn.tree import DecisionTreeClassifier   ## Tree 분류모델
from sklearn.ensemble import RandomForestClassifier  ## RandomForest
from sklearn.ensemble import RandomForestRegressor   ## RandomForest
from sklearn.linear_model import LogisticRegression  ## Logistic

from sklearn.metrics import accuracy_score       ## 정확도 검증

import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State

In [108]:
df = pd.read_csv('https://gist.githubusercontent.com/michhar/2dfd2de0d4f8727f873422c5d959fff5/raw/fa71405126017e6a37bea592440b4bee94bf7b9e/titanic.csv')

In [109]:
dt = df.copy()

In [110]:
## Na 데이터 채우기
dt.Age.fillna(dt.Age.mean(),inplace=True)
dt.Embarked.fillna('N',inplace=True)
dt.Cabin.fillna('N',inplace=True)

In [111]:
## object 
## Sex ,Embarked, new_Cabin
gender_enc = LabelEncoder()
gender_enc.fit(dt.Sex)
gender = gender_enc.transform(dt.Sex)

embark_enc = OneHotEncoder()
embark_enc.fit(dt.Embarked.values.reshape(-1,1))
embark = embark_enc.transform(dt.Embarked.values.reshape(-1,1)).toarray()

cabin_enc = OneHotEncoder()
cabin_enc.fit(dt.Cabin.str.slice(0,1).values.reshape(-1,1))
cabin = cabin_enc.transform(dt.Cabin.str.slice(0,1).values.reshape(-1,1)).toarray()

pclass_enc = OneHotEncoder()
pclass_enc.fit(dt.Pclass.values.reshape(-1,1))
pclass = pclass_enc.transform(dt.Pclass.values.reshape(-1,1)).toarray()

enc_data = np.concatenate([gender.reshape(-1,1),embark, cabin,pclass],axis = 1)
adult = np.where(df.Name.str.contains('Mr','Miss'),1,0)

In [112]:
feature_col = ['SibSp','Parch','Age']
dep_col = ['Survived']
X = pd.concat([dt[feature_col],pd.DataFrame(enc_data),pd.Series(adult)], axis = 1)
y = dt[dep_col]

In [99]:
# SVC
Cs = np.linspace(0.1,10,10)
gammas = np.linspace(0.001,2,10)
kernels = ['linear', 'rbf', 'sigmoid']
params = { 'kernel': kernels, 'C' : Cs, 'gamma' : gammas}
clf = GridSearchCV( SVC(), params, cv = 5, n_jobs = -1)
clf.fit(X,y)
svc_clf = SVC(**clf.best_params_)
svc_clf.fit(X,y)
svc_clf.score(X,y)

KeyboardInterrupt: 

In [68]:
# Logistic
params = { 'penalty' : ['l1', 'l2'], 
         'C' : np.linspace(0.1,2,10)}
clf = GridSearchCV( LogisticRegression(), params, cv = 5, n_jobs = -1)
clf.fit(X,y)
clf.best_score_
lor_clf = LogisticRegression(**clf.best_params_)
lor_clf.fit(X,y)
lor_clf.score(X,y)

0.8159371492704826

In [69]:
# Tree
max_depth = np.arange(3,10)
min_samples_split = np.arange(2,5)
criterion = np.linspace(0.1,20,20)
params = { 'criterion' : ['gini','entropy'], 'max_depth' : max_depth, 'min_samples_split' : min_samples_split}
clf = GridSearchCV( DecisionTreeClassifier(), params, cv = 5, n_jobs = -1)
clf.fit(X,y)
clf.best_score_
dt_clf = DecisionTreeClassifier(**clf.best_params_)
dt_clf.fit(X,y)
dt_clf.score(X,y)

0.8664421997755332

In [70]:
# LinearRegression
lr_clf= LinearRegression()
lr_clf.fit(X,y)
lr_clf.score(X,y)

0.41312435974266326

In [71]:
# RandomForestClassifier
rf_clf = RandomForestClassifier()
rf_clf.fit(X,y)
rf_clf.score(X,y)

0.9326599326599326

# ==========================================

# **Dash**

In [124]:
df = pd.read_csv('https://gist.githubusercontent.com/michhar/2dfd2de0d4f8727f873422c5d959fff5/raw/fa71405126017e6a37bea592440b4bee94bf7b9e/titanic.csv')
dt = df.copy()
## Na 데이터 채우기
dt.Age.fillna(dt.Age.mean(),inplace=True)
dt.Embarked.fillna('N',inplace=True)
dt.Cabin.fillna('N',inplace=True)
## object 
## Sex ,Embarked, new_Cabin
gender_enc = LabelEncoder()
gender_enc.fit(dt.Sex)
gender = gender_enc.transform(dt.Sex)

embark_enc = OneHotEncoder()
embark_enc.fit(dt.Embarked.values.reshape(-1,1))
embark = embark_enc.transform(dt.Embarked.values.reshape(-1,1)).toarray()

cabin_enc = OneHotEncoder()
cabin_enc.fit(dt.Cabin.str.slice(0,1).values.reshape(-1,1))
cabin = cabin_enc.transform(dt.Cabin.str.slice(0,1).values.reshape(-1,1)).toarray()

pclass_enc = OneHotEncoder()
pclass_enc.fit(dt.Pclass.values.reshape(-1,1))
pclass = pclass_enc.transform(dt.Pclass.values.reshape(-1,1)).toarray()

enc_data = np.concatenate([gender.reshape(-1,1),embark, cabin,pclass],axis = 1)
adult = np.where(df.Name.str.contains('Mr','Miss'),1,0)

feature_col = ['SibSp','Parch','Age']
dep_col = ['Survived']
X = pd.concat([dt[feature_col],pd.DataFrame(enc_data),pd.Series(adult)], axis = 1)
y = dt[dep_col]

In [125]:
gender = dcc.Dropdown(
    id = 'gender',
    options = [{'label' : 'female', 'value' : 'female'},
              {'label' : 'male', 'value' : 'male'}],
    placeholder = '성별을 입력하시오'
)
embark = dcc.Dropdown(
    id = 'embark',
    options = [{'label' : 'Cherbourg', 'value' : 'C'},
              {'label' : 'Queenstown', 'value' : 'Q'},
              {'label' : 'Southampton', 'value' :'S'}],
    placeholder = '정박지를 입력하시오'
)
pclass = dcc.Dropdown(
    id = 'pclass',
    options = [{'label' : '1등석', 'value' : 1},
              {'label' : '2등석', 'value' : 2},
              {'label' : '3등석', 'value' : 3}],
    placeholder = '선실종류를 선택하시오'
)
sibsp = dcc.Input(
    id = 'sibsp',
    type = 'number'
)
parch = dcc.RadioItems(
    id = 'parch',
    options = [{'label' : '0', 'value' : 0},
              {'label' : '1', 'value' : 1},
              {'label' : '2', 'value' : 2},
              {'label' : '3', 'value' : 3},
              {'label' : '4', 'value' : 4},
              {'label' : '5', 'value' : 5},
              {'label' : '6', 'value' : 6}]
)
cabin = dcc.Dropdown(
    id = 'cabin',
    options = [{'label' : i , 'value' : i} for i in dt.Cabin.str.slice(0,1).unique()]
)
age = dcc.Input(
    id = 'age',
    type = 'text'
)
button = html.Button(
    '생존확인',
    id = 'button',
    n_clicks = 0
)

In [126]:
app = dash.Dash(__name__)
app.layout = html.Div(
    [
        gender,
        embark,
        pclass,
        sibsp,
        parch,
        cabin,
        age,
        button,
        html.Div(
            id = 'html-output'
        )
    ]
)

In [127]:
@app.callback(
    Output('html-output','children'),
    Input('button','n_clicks'),
    State('embark', 'value'),
    State('gender','value'),
    State('pclass','value'),
    State('sibsp','value'),
    State('parch','value'),
    State('cabin','value'),
    State('age','value')
)
def html_output(n, embark, gender, pclass, sibsp, parch, cabin,age):
    embark = np.array(embark).reshape(-1,1)
    embark = embark_enc.transform(embark).toarray()
    
    sibsp = sibsp
    parch = parch
    age = age
    
    gender = gender_enc.transform([gender])
    
    pclass = np.array(pclass).reshape(-1,1)
    pclass = pclass_enc.transform(pclass).toarray()
    
    cabin = np.array(cabin).reshape(-1,1)
    cabin = cabin_enc.transform(cabin).toarray()
    
    x = 
    
    return html.H1(str(parch))

In [138]:
sibsp = 2
parch = 3
age = 30


sibsp = np.array([sibsp]).reshape(-1,1)
parch = np.array([parch]).reshape(-1,1)
age = np.array([age]).reshape(-1,1)
embark = [[0,1,0]]
cabin
pclass


np.concatenate([sibsp,parch,age,gender, embark], axis=1)



array([[2, 3, 0, 1, 0]])

In [130]:
X

,SibSp,Parch,Age,0,1,2,3,4,5,6,...,8,9,10,11,12,13,14,15,16,0
0,1,0,22.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1
1,1,0,38.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
2,0,0,26.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
3,1,0,35.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
4,0,0,35.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,0,27.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0
887,0,0,19.000000,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
888,1,2,29.699118,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
889,0,0,26.000000,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1


In [128]:
app.run_server(host = '127.0.0.1',port='9999')

Dash is running on http://127.0.0.1:9999/

Dash is running on http://127.0.0.1:9999/

Dash is running on http://127.0.0.1:9999/

Dash is running on http://127.0.0.1:9999/

Dash is running on http://127.0.0.1:9999/

Dash is running on http://127.0.0.1:9999/

Dash is running on http://127.0.0.1:9999/

Dash is running on http://127.0.0.1:9999/

Dash is running on http://127.0.0.1:9999/

Dash is running on http://127.0.0.1:9999/

Dash is running on http://127.0.0.1:9999/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:9999/ (Press CTRL+C to quit)
127.0.0.1 - - [25/May/2021 17:10:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 17:10:22] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 17:10:22] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 17:10:22] "GET /_favicon.ico?v=1.20.0 HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\admin\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\admin\Anaconda3\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_d

127.0.0.1 - - [25/May/2021 17:10:22] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [25/May/2021 17:10:28] "POST /_dash-update-component HTTP/1.1" 200 -
